**요약**
- 미세조정된 SegFormer를 `train_source_image`, `train_target_image`, `val_source_image`에 다시 한 번 미세조정합니다.

<br>

**Inputs:**
- `dir_data`: 데이터가 있는 디렉토리
- `dir_ckpt`: 학습된 모델을 저장할 디렉토리
- `path_ckpt`: 수도 레이블링을 생성한 미세조정된 SegFormer 모델의 체크포인트

<br>

**Outputs**:
- f`{dir_ckpt}/1695950569_pl`: 수도 레이블링까지 학습한 모델 체크포인트

In [1]:
dir_data = '../data'
dir_ckpt = '../ckpt'
path_ckpt = '../ckpt/1695288341/last_ckpt.bin'

In [2]:
import sys
sys.path.append('../')

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import pandas as pd

from datetime import datetime
from torch.utils.data import DataLoader
from transformers import SegformerImageProcessor

from segformers.utils import seed_all, print_env
from segformers.config import pseudo_labeling_config
from segformers.trainer import Trainer
from segformers.networks import SegFormer
from segformers.datasets import SourceDataset
from segformers.transforms import augmentation_pl


/home/dongjin/miniconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b5-finetuned-cityscapes-1024-1024 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([19, 768, 1, 1]) in the checkpoint and torch.Size([13, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([19]) in the checkpoint and torch.Size([13]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-large-ci

In [3]:
print_env()

========== System Information ==========
DATE : 2023-10-04
Pyton Version : 3.8.17
PyTorch Version : 1.13.0
OS : Linux 5.4.0-155-generic
CPU spec : x86_64
RAM spec : 503.73 GB
Device 0:
Name: NVIDIA A100-SXM4-40GB
Total Memory: 40536.1875 MB
Driver Version: 470.199.02
Device 1:
Name: NVIDIA A100-SXM4-40GB
Total Memory: 40536.1875 MB
Driver Version: 470.199.02
Device 2:
Name: NVIDIA A100-SXM4-40GB
Total Memory: 40536.1875 MB
Driver Version: 470.199.02
Device 3:
Name: NVIDIA DGX Display
Total Memory: 3911.875 MB
Driver Version: 470.199.02
Device 4:
Name: NVIDIA A100-SXM4-40GB
Total Memory: 40536.1875 MB
Driver Version: 470.199.02


In [4]:
seed_all(pseudo_labeling_config['seed'])
run_id = int(datetime.timestamp(datetime.now()))
pseudo_labeling_config['run_id'] = run_id
pseudo_labeling_config['dir_ckpt'] = os.path.join(dir_ckpt, str(run_id)+'_pl')
os.makedirs(pseudo_labeling_config['dir_ckpt'])

state_dict = torch.load(path_ckpt, map_location='cpu')
model = SegFormer
model.load_state_dict(state_dict['model_state_dict'])


<All keys matched successfully>

In [5]:
image_processor = SegformerImageProcessor(
    image_mean=[0.485, 0.456, 0.406],
    image_std=[0.229, 0.224, 0.225],
)

train_dataset = SourceDataset(
    root=dir_data,
    csv_file='full_pl.csv',
    image_processor=image_processor,
    transform=augmentation_pl,
    is_training=True
)

valid_dataset = SourceDataset(
    root=dir_data,
    csv_file='full_pl.csv',
    image_processor=image_processor,
    transform=None,
    is_training=True
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=4)

In [6]:
trainer = Trainer(model, pseudo_labeling_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dongjinlee. Use `wandb login --relogin` to force relogin


In [7]:
trainer.fit(train_loader, valid_loader)

Epoch: 1.0000 | LR: 0.0000 | train_Loss: 0.2918 | train_mIoU: 0.7168 | valid_Loss: 0.5104 | valid_mIoU: 0.6013 |
Epoch: 2.0000 | LR: 0.0000 | train_Loss: 0.1346 | train_mIoU: 0.7343 | valid_Loss: 0.1607 | valid_mIoU: 0.6431 |
Epoch: 3.0000 | LR: 0.0000 | train_Loss: 0.0847 | train_mIoU: 0.7441 | valid_Loss: 0.1402 | valid_mIoU: 0.6463 |
Epoch: 4.0000 | LR: 0.0000 | train_Loss: 0.0756 | train_mIoU: 0.7526 | valid_Loss: 0.1377 | valid_mIoU: 0.6489 |
Epoch: 5.0000 | LR: 0.0000 | train_Loss: 0.0729 | train_mIoU: 0.7586 | valid_Loss: 0.1318 | valid_mIoU: 0.6527 |


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁▃▅▆█
LR,▁▃▅▆█
train_Loss,█▃▁▁▁
train_mIoU,▁▄▆▇█
valid_Loss,█▂▁▁▁
valid_mIoU,▁▇▇▇█
Epoch,5
LR,1e-05
train_Loss,0.07291
train_mIoU,0.75862
valid_Loss,0.13184
